In [29]:
import pandas as pd
import bs4
import requests
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [30]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')
go_df = pd.read_csv('./go_annotations.csv')
kegg_df = pd.read_csv('./kegg_annotations.csv')
ec_df = pd.read_csv('./ec_annotations.csv')

In [31]:
go_df.head()

,GOs,GOs_description,GOs_definition,GOs_obsolete
0,-,-,-,-
1,GO:0000001,mitochondrion inheritance,"The distribution of mitochondria, including th...",False
2,GO:0000002,mitochondrial genome maintenance,The maintenance of the structure and integrity...,False
3,GO:0000003,reproduction,The production of new individuals that contain...,False
4,GO:0000011,vacuole inheritance,The distribution of vacuoles into daughter cel...,False


In [32]:
go_df['GOs_description'].loc[go_df['GOs'] == 'GO:0000001'].values[0]

'mitochondrion inheritance'

In [33]:
kegg_df.head()

,KEGG_ko,KEGG_ko_description
0,-,-
1,ko:K00002,alcohol dehydrogenase (NADP+)
2,ko:K00006,glycerol-3-phosphate dehydrogenase (NAD+)
3,ko:K00011,aldehyde reductase
4,ko:K00012,UDPglucose 6-dehydrogenase


In [34]:
ec_df.head()

,EC,EC_description
0,-,-
1,1.1.1.1,alcohol dehydrogenase
2,1.1.1.105,all-trans-retinol dehydrogenase (NAD(+))
3,1.1.1.141,15-hydroxyprostaglandin dehydrogenase (NAD(+))
4,1.1.1.178,3-hydroxy-2-methylbutyryl-CoA dehydrogenase


In [35]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [36]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [37]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [38]:
def get_GO_info(go_term):
    
    name = go_df['GOs_description'].loc[go_df['GOs'] == go_term].values[0]
    
    definition = go_df['GOs_definition'].loc[go_df['GOs'] == go_term].values[0]
    
    obsolete = go_df['GOs_obsolete'].loc[go_df['GOs'] == go_term].values[0]
    
    return name, definition, obsolete

In [39]:
get_GO_info('GO:0000001')

('mitochondrion inheritance',
 'The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton.',
 'False')

In [40]:
def get_KEGG_info(term):
    return kegg_df['KEGG_ko_description'].loc[kegg_df['KEGG_ko'] == term].values[0]

In [41]:
get_KEGG_info('ko:K00002')

'alcohol dehydrogenase (NADP+)'

In [42]:
def get_EC_info(term):
    return ec_df['EC_description'].loc[ec_df['EC'] == term].values[0]

In [43]:
get_EC_info('1.1.1.141')

'15-hydroxyprostaglandin dehydrogenase (NAD(+))'

In [44]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annotation, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [45]:


# response = requests.get('https://rest.kegg.jp/get/ko:K19607')

In [46]:
# test = response.text.split('\n')

In [47]:
# name = re.search(r'NAME\s*[\w \-]*', response.text).group()

In [48]:
# re.split(r'\s\s+', name)

In [49]:
# name

In [50]:
# test[:10]

In [51]:
# r = requests.get("https://www.ebi.ac.uk/enzymeportal/rest/enzymes/2.7.11.1")
# r.json()['enzymeName']

In [52]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [53]:
lldf_nn3 = pd.read_csv('../embedding/test_nn3_leiden_label_df_round_1_rearranged.csv')

In [54]:
lldf_nn3['leiden_label_full'].unique()

array([11, 32,  1, 30, 44, 18, 22, 31,  3, 34, 42, 29,  7,  0,  2, 23, 15,
       10, 21,  9,  4, 27, 19,  6, 20, 14, 26, 25, 50,  8, 28, 35, 13, 38,
       36, 33, 12, 39,  5, 40, 17, 16, 41, 53, 43, 45, 48, 37, 24, 46, 47,
       52, 49, 51])

In [55]:
# %pdb

In [56]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'full', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

  0%|          | 0/54 [00:00<?, ?it/s]

100%|██████████| 54/54 [00:25<00:00,  2.14it/s]


In [57]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [58]:
enrich_full_nn3.head()

,module,term,info,fold_change,bonferroni
0,0.0,Q,"Secondary metabolites biosynthesis, transport ...",2.544908,0.000019
1,0.0,T,Signal transduction mechanisms,1.488057,0.000606
2,0.0,I,Lipid transport and metabolism,1.872586,0.038869
3,0.0,ko:K21867,potassium channel,3.182788,0.045850
4,1.0,ko:K10418,dynein light chain LC8-type,23.958029,0.002061


In [59]:
enrich_full_nn3['module'].unique()

array([ 0.,  1.,  3.,  4.,  9., 11., 12., 13., 14., 16., 17., 18., 20.,
       24., 28., 29., 31., 32., 34., 35., 36., 37., 38., 41., 45., 46.,
       47.])

In [60]:
lldf_nn3['leiden_label_full'].unique()

array([11, 32,  1, 30, 44, 18, 22, 31,  3, 34, 42, 29,  7,  0,  2, 23, 15,
       10, 21,  9,  4, 27, 19,  6, 20, 14, 26, 25, 50,  8, 28, 35, 13, 38,
       36, 33, 12, 39,  5, 40, 17, 16, 41, 53, 43, 45, 48, 37, 24, 46, 47,
       52, 49, 51])

In [61]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,Unnamed: 0,TTHERM_ID,leiden_label_full
24619,24619,TTHERM_01386050,29


end here. Return to embedding notebook!